In [1]:
from datasets import load_dataset
import random
from transformers import RobertaTokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import torch
import nltk

2024-08-21 03:52:38.768936: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-21 03:52:38.769059: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-21 03:52:38.924334: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Load the CodeSearchNet dataset for each language
dataset_java = load_dataset('code_search_net', 'java')['train']
dataset_python = load_dataset('code_search_net', 'python')['train']
dataset_go = load_dataset('code_search_net', 'go')['train']

The repository for code_search_net contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/code_search_net.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split:   0%|          | 0/454451 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/26909 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/15328 [00:00<?, ? examples/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Generating test split:   0%|          | 0/14291 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/14242 [00:00<?, ? examples/s]

In [3]:
# Function to sample 100 examples
def sample_data(dataset, num_samples=1000):
    sampled_indices = random.sample(range(len(dataset)), num_samples)
    return dataset.select(sampled_indices)

# Sample 100 pairs from each language
sampled_java = sample_data(dataset_java)
sampled_python = sample_data(dataset_python)
sampled_go = sample_data(dataset_go)

# Function to split data into train, validation, and test sets
def train_val_test_split_data(dataset, val_size=0.1, test_size=0.1):
    train_size = 1 - (val_size + test_size)
    # Shuffle and split the dataset into train and temporary datasets
    train_dataset = dataset.shuffle(seed=42).select(range(int(train_size * len(dataset))))
    temp_dataset = dataset.shuffle(seed=42).select(range(int(train_size * len(dataset)), len(dataset)))
    # Split the temporary dataset into validation and test datasets
    split = int(len(temp_dataset) * (test_size / (test_size + val_size)))
    val_dataset = temp_dataset.select(range(len(temp_dataset) - split))
    test_dataset = temp_dataset.select(range(len(temp_dataset) - split, len(temp_dataset)))
    return train_dataset, val_dataset, test_dataset

# Split the data into train, validation, and test sets for each language
train_java, val_java, test_java = train_val_test_split_data(sampled_java)
train_python, val_python, test_python = train_val_test_split_data(sampled_python)
train_go, val_go, test_go = train_val_test_split_data(sampled_go)

# Load the tokenizer for CodeT5+
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5p-220m')

# Function to preprocess the datasets
def preprocess_function(examples):
    # Use 'func_code_string' for the code and 'func_documentation_string' for the documentation
    inputs = examples['func_code_string']
    targets = examples['func_documentation_string']
    # Tokenize the inputs and targets
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    # Tokenize the targets (docstrings) and add them as labels
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length").input_ids
    # Assign the labels to the model inputs
    model_inputs["labels"] = labels
    return model_inputs

# Preprocess the datasets
train_java = train_java.map(preprocess_function, batched=True)
val_java = val_java.map(preprocess_function, batched=True)
test_java = test_java.map(preprocess_function, batched=True)

train_python = train_python.map(preprocess_function, batched=True)
val_python = val_python.map(preprocess_function, batched=True)
test_python = test_python.map(preprocess_function, batched=True)

train_go = train_go.map(preprocess_function, batched=True)
val_go = val_go.map(preprocess_function, batched=True)
test_go = test_go.map(preprocess_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

...

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [6]:
# Import necessary libraries
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments
import nltk
import torch
from nltk.translate.bleu_score import SmoothingFunction

# Load the pre-trained CodeT5+ model
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5p-220m')

# Download necessary resources for nltk BLEU
nltk.download('punkt')

# Function to fine-tune the model
def fine_tune(model, train_dataset, val_dataset, output_dir):
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        eval_strategy="epoch",  # Updated to 'eval_strategy'
        save_steps=1000,
        save_total_limit=2,
        logging_dir='./logs',
    )
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
    )
    trainer.train()
    return model

# Function to evaluate the model using BLEU score with smoothing
def evaluate_model(model, test_dataset, device):
    model.to(device)  # Ensure the model is on the correct device
    test_results = []

    for example in test_dataset:
        # Tokenize the input and move it to the appropriate device
        inputs = tokenizer(example['func_code_string'], return_tensors="pt", max_length=512, truncation=True).to(device)
        # Generate predictions with explicit max_new_tokens
        outputs = model.generate(**inputs, max_new_tokens=100)
        prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
        # Reference remains on the CPU (as it's just a string comparison)
        reference = example['func_documentation_string']
        test_results.append({"prediction": prediction, "reference": reference})

    # Extract predictions and references
    predictions = [result["prediction"] for result in test_results]
    references = [nltk.word_tokenize(result["reference"]) for result in test_results]
    tokenized_predictions = [nltk.word_tokenize(pred) for pred in predictions]

    # Compute BLEU scores with smoothing
    smoothie = SmoothingFunction().method4
    bleu = nltk.translate.bleu_score.corpus_bleu([[ref] for ref in references], tokenized_predictions, smoothing_function=smoothie)
    return {"bleu": bleu}

# Define the device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Fine-tune and evaluate sequentially on each language

# Step 1: Fine-tune on Java
model = fine_tune(model, train_java, val_java, './results_java')
java_results_after_java = evaluate_model(model, test_java, device)
python_results_after_java = evaluate_model(model, test_python, device)
go_results_after_java = evaluate_model(model, test_go, device)

# Step 2: Fine-tune on Python
model = fine_tune(model, train_python, val_python, './results_python')
java_results_after_python = evaluate_model(model, test_java, device)
python_results_after_python = evaluate_model(model, test_python, device)
go_results_after_python = evaluate_model(model, test_go, device)

# Step 3: Fine-tune on Go
model = fine_tune(model, train_go, val_go, './results_go')
java_results_after_go = evaluate_model(model, test_java, device)
python_results_after_go = evaluate_model(model, test_python, device)
go_results_after_go = evaluate_model(model, test_go, device)

# Display results
print("Results after training on Java:")
print("Java Test Results:", java_results_after_java)
print("Python Test Results:", python_results_after_java)
print("Go Test Results:", go_results_after_java)

print("\nResults after training on Python:")
print("Java Test Results:", java_results_after_python)
print("Python Test Results:", python_results_after_python)
print("Go Test Results:", go_results_after_python)

print("\nResults after training on Go:")
print("Java Test Results:", java_results_after_go)
print("Python Test Results:", python_results_after_go)
print("Go Test Results:", go_results_after_go)


config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/446M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

...

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,No log,3.507134
2,No log,2.986073
3,No log,2.823178


Epoch,Training Loss,Validation Loss
1,No log,1.097229
2,No log,0.615844
3,No log,0.483468


Epoch,Training Loss,Validation Loss
1,No log,0.532767
2,No log,0.438091
3,No log,0.430539


Results after training on Java:
Java Test Results: {'bleu': 0.10915195799493187}
Python Test Results: {'bleu': 0.272069463064233}
Go Test Results: {'bleu': 0.026967006049624365}

Results after training on Python:
Java Test Results: {'bleu': 0.0009397340718526295}
Python Test Results: {'bleu': 0.6164936346733784}
Go Test Results: {'bleu': 0.006472781283670101}

Results after training on Go:
Java Test Results: {'bleu': 0.01397301280093913}
Python Test Results: {'bleu': 0.5525798712081508}
Go Test Results: {'bleu': 0.1832540470814492}


In [7]:
#1000 codes t5+ no ewc